In [44]:
# import libraries
import pandas as pd
import numpy as np

In [45]:
# Load files to be merged

values_to_load = "Resources/saveris_values_complete.csv"
alarms_to_load = "Resources/saveris_alarms_clean.csv"

values_df = pd.read_csv(values_to_load)
alarms_df = pd.read_csv(alarms_to_load)


alarms_df.shape

(7220, 7)

In [46]:
values_df.shape

(31648, 11)

In [47]:
# Join Data frames 
df_alarm = pd.merge(alarms_df, values_df, on='date', how='right')
print(df_alarm.shape)
df_alarm.head()
# df_dataset[25000:25050]

(32985, 17)


,date,location,sensor_name,status,alarm_type,limit,value,platform_1,platform_2,deepfreezer_1,deepfreezer_2,freezer_1,freezer_2,freezer_3,freezer_4,cooling_1,cooling_2
0,2021-06-24 16:45:00,Freezer,freezer_1,0.0,recovery,-19.0,-19.3,3.7,1.4,-20.0,-22.9,-19.3,-19.1,-18.1,-20.8,3.9,3.9
1,2021-06-24 16:45:00,Freezer,freezer_2,0.0,recovery,-19.0,-19.1,3.7,1.4,-20.0,-22.9,-19.3,-19.1,-18.1,-20.8,3.9,3.9
2,2021-06-24 17:00:00,Freezer,freezer_1,0.0,recovery,-19.0,-20.0,2.3,0.1,-23.0,-24.0,-20.2,-19.9,-20.0,-21.6,3.4,3.5
3,2021-06-24 17:15:00,NaN,NaN,NaN,NaN,NaN,NaN,2.4,0.6,-24.8,-24.6,-20.6,-20.4,-20.9,-21.9,3.2,3.3
4,2021-06-24 17:30:00,Deepfreezer,deepfreezer_1,0.0,recovery,-25.0,-25.9,2.4,0.6,-25.9,-25.0,-20.9,-20.8,-21.2,-22.1,3.1,3.2


### We are going to use this df to produce our final alarm dataset. This one will be able to tell us which one of the 10 sensors was the one that activated the alarm.

In [48]:
# First we are dropping 'sensor_name' column
df_alarm = df_alarm.drop('sensor_name', inplace=False, axis=1)

# Drop NaN values
df_alarm = df_alarm.dropna()

print(df_alarm.shape)
df_alarm.head()

(7123, 16)


,date,location,status,alarm_type,limit,value,platform_1,platform_2,deepfreezer_1,deepfreezer_2,freezer_1,freezer_2,freezer_3,freezer_4,cooling_1,cooling_2
0,2021-06-24 16:45:00,Freezer,0.0,recovery,-19.0,-19.3,3.7,1.4,-20.0,-22.9,-19.3,-19.1,-18.1,-20.8,3.9,3.9
1,2021-06-24 16:45:00,Freezer,0.0,recovery,-19.0,-19.1,3.7,1.4,-20.0,-22.9,-19.3,-19.1,-18.1,-20.8,3.9,3.9
2,2021-06-24 17:00:00,Freezer,0.0,recovery,-19.0,-20.0,2.3,0.1,-23.0,-24.0,-20.2,-19.9,-20.0,-21.6,3.4,3.5
4,2021-06-24 17:30:00,Deepfreezer,0.0,recovery,-25.0,-25.9,2.4,0.6,-25.9,-25.0,-20.9,-20.8,-21.2,-22.1,3.1,3.2
5,2021-06-24 17:30:00,Deepfreezer,0.0,recovery,-25.0,-25.0,2.4,0.6,-25.9,-25.0,-20.9,-20.8,-21.2,-22.1,3.1,3.2


In [49]:
# Create a new column. If column 'value' is equal to one of the values from the "sensor columns",
# set the name to that sensor column.

conditions = [df_alarm.value.eq(df_alarm.platform_1), 
              df_alarm.value.eq(df_alarm.platform_2), df_alarm.value.eq(df_alarm.deepfreezer_1), 
              df_alarm.value.eq(df_alarm.deepfreezer_2), df_alarm.value.eq(df_alarm.freezer_1),
             df_alarm.value.eq(df_alarm.freezer_2), df_alarm.value.eq(df_alarm.freezer_3),
             df_alarm.value.eq(df_alarm.freezer_4), df_alarm.value.eq(df_alarm.cooling_1),
             df_alarm.value.eq(df_alarm.cooling_2)]

choices = ['platform_1', 'platform_2', 'deepfreezer_1', 'deepfreezer_2', 'freezer_1', 'freezer_2', 'freezer_3', 'freezer_4', 'cooling_1', 'cooling_2']

df_alarm['sensor_alarm'] = np.select(conditions, choices)


In [50]:
df_alarm.head(50)

,date,location,status,alarm_type,limit,value,platform_1,platform_2,deepfreezer_1,deepfreezer_2,freezer_1,freezer_2,freezer_3,freezer_4,cooling_1,cooling_2,sensor_alarm
0,2021-06-24 16:45:00,Freezer,0.0,recovery,-19.0,-19.3,3.7,1.4,-20.0,-22.9,-19.3,-19.1,-18.1,-20.8,3.9,3.9,freezer_1
1,2021-06-24 16:45:00,Freezer,0.0,recovery,-19.0,-19.1,3.7,1.4,-20.0,-22.9,-19.3,-19.1,-18.1,-20.8,3.9,3.9,freezer_2
2,2021-06-24 17:00:00,Freezer,0.0,recovery,-19.0,-20.0,2.3,0.1,-23.0,-24.0,-20.2,-19.9,-20.0,-21.6,3.4,3.5,freezer_3
4,2021-06-24 17:30:00,Deepfreezer,0.0,recovery,-25.0,-25.9,2.4,0.6,-25.9,-25.0,-20.9,-20.8,-21.2,-22.1,3.1,3.2,deepfreezer_1
5,2021-06-24 17:30:00,Deepfreezer,0.0,recovery,-25.0,-25.0,2.4,0.6,-25.9,-25.0,-20.9,-20.8,-21.2,-22.1,3.1,3.2,deepfreezer_2
6,2021-06-24 17:45:00,Platform,1.0,deficit,1.0,0.6,2.4,0.6,-26.4,-25.4,-21.0,-21.0,-21.5,-22.2,3.1,3.2,platform_2
12,2021-06-24 19:15:00,Platform,1.0,deficit,1.0,0.8,0.8,-0.9,-27.2,-26.2,-21.1,-21.0,-22.2,-22.4,3.0,3.1,platform_1
13,2021-06-24 19:30:00,Platform,0.0,recovery,1.0,1.1,1.1,-0.5,-27.6,-26.3,-21.1,-20.9,-22.3,-22.4,3.0,3.1,platform_1
19,2021-06-24 21:00:00,Platform,1.0,deficit,1.0,0.7,0.7,-1.1,-27.7,-26.9,-20.8,-20.7,-21.6,-22.3,3.4,3.2,platform_1
20,2021-06-24 21:15:00,Platform,0.0,recovery,1.0,1.1,1.1,-0.6,-27.7,-27.0,-20.8,-20.8,-21.7,-22.3,3.5,3.3,platform_1


In [51]:
df_alarm.dtypes

date              object
location          object
status           float64
alarm_type        object
limit            float64
value            float64
platform_1       float64
platform_2       float64
deepfreezer_1    float64
deepfreezer_2    float64
freezer_1        float64
freezer_2        float64
freezer_3        float64
freezer_4        float64
cooling_1        float64
cooling_2        float64
sensor_alarm      object
dtype: object

In [52]:
df_alarm.to_csv('Resources/saveris_alarms_complete.csv', index=False)